# Classificação de processos de homicídios
### Importando dataset gerado via webcrawler

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("resultado_pesquisa_2021_07_10_23_49_56.csv", sep=";", encoding = "ISO-8859-1")

In [3]:
dataset.head()

,classe,assunto,magistrado,comarca,foro,vara,data de disponibilizacao,texto
0,Ação Penal de Competência do Júri,Homicídio Simples,Bruno Nascimento Troccoli,Mongaguá,Foro de Mongaguá,2ª Vara,09/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
1,Ação Penal - Procedimento Sumário,Crimes de Trânsito,FABIO ANTONIO CAMARGO DANTAS,Jales,Foro de Jales,1ª Vara Criminal,09/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
2,Ação Penal de Competência do Júri,Homicídio Qualificado,EMERSON TADEU PIRES DE CAMARGO,Sorocaba,Foro de Sorocaba,Vara do Júri/Execuções,09/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
3,Ação Penal - Procedimento Ordinário,Furto,EDUARDO DE FRANCA HELENE,São José dos Campos,Foro de São José dos Campos,3ª Vara Criminal,08/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
4,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Fabio Pando de Matos,SÃO PAULO,Foro Central Criminal Barra Funda,10ª Vara Criminal,08/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...


### Limpando os dados e removendo informações desnecessárias

In [4]:
non_null_ds = dataset.dropna()

df = non_null_ds[['assunto','magistrado','texto']].copy()
df = df[df.assunto.str.contains('Homicídio')]
df = df[df.texto.str.contains('Vistos.')]
df.texto.head()

0     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
2     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
7     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
9     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
10    TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
Name: texto, dtype: object

In [5]:
df['texto'] = df['texto'].map(lambda x: str(x).split('Vistos.')[-1].split('Intime-se')[0])

In [6]:
df.head()

,assunto,magistrado,texto
0,Homicídio Simples,Bruno Nascimento Troccoli,Os acusados JOSIMAR FERREIRA DA FONSECA e ...
2,Homicídio Qualificado,EMERSON TADEU PIRES DE CAMARGO,"O réu ARISTIDES GOMES DOS SANTOS, qualif..."
7,Homicídio Simples,GABRIEL ALVES BUENO PEREIRA,"LUIZ FERNANDO DOS SANTOS PEREIRA, qualificad..."
9,Homicídio Simples,ERICA PEREIRA DE SOUSA,SIDNEY DIAS DE SOUSA foi denunciado ...
10,Homicídio Simples,Paulo Rogério Santos Pinheiro,"EDNALDO FREITAS DE OLIVEIRA, qualificado nos..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23340 entries, 0 to 65619
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   assunto     23340 non-null  object
 1   magistrado  23340 non-null  object
 2   texto       23340 non-null  object
dtypes: object(3)
memory usage: 729.4+ KB


In [8]:
df.describe()

,assunto,magistrado,texto
count,23340,23340,23340
unique,3,1219,22781
top,Homicídio Simples,Fernando Martinho de Barros Penteado,
freq,14086,428,8


In [9]:
df = df[:2000]

In [10]:
df['culpado'] = df.texto.str.contains('cumprimento de pena')

In [11]:
print('Número de casos por classe') 
df.groupby('assunto').size()

Número de casos por classe


assunto
Homicídio Privilegiado    149
Homicídio Qualificado     933
Homicídio Simples         918
dtype: int64

### Aplicando NLP

#### Removendo stop words

In [12]:
import ssl
try:
     _create_unverified_https_context =     ssl._create_unverified_context
except AttributeError:
     pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [13]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
pt_br_stopwords = stopwords.words('portuguese')

print(pt_br_stopwords)

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estiv

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sam.pereira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Importando Scikit lib

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='ISO-8859-1', ngram_range=(1, 2), stop_words=pt_br_stopwords)
features = tfidf.fit_transform(df.texto).toarray()

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

text_input = features
text_output = df['assunto']

le = LabelEncoder()
text_output = le.fit_transform(text_output)

X_train, X_test, y_train, y_test = train_test_split(text_input,text_output,test_size=0.3)

### Criando modelos de classificação

#### KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier

# Instantiate learning model (k = 3)
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Fitting the model
knn_classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = knn_classifier.predict(X_test)

In [18]:
accuracy_knn = accuracy_score(y_test, y_pred)*100
print('KNN accuracy = ' + str(round(accuracy_knn, 1)) + ' %.')
print("K-Nearest Neighbors (KNN): accuracy = %3.1f %%" % accuracy_knn)


KNN accuracy = 54.5 %.
K-Nearest Neighbors (KNN): accuracy = 54.5 %


#### Decision tree

In [19]:
from pyspark.ml.classification import DecisionTreeClassifier

#### Naive Bayes

In [20]:
from pyspark.ml.classification import NaiveBayes